In [ ]:
import tkinter as tk
from tkinter import messagebox
import sounddevice as sd
import numpy as np
import librosa
import joblib
from keras.models import load_model
from scipy.io.wavfile import write
import os

# Load model, scaler, and label classes
model = load_model('my_model.keras')
scaler = joblib.load('scaler.save')
label_classes = np.load('classes.npy')

# ========== Feature Extraction ==========
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    audio, sr = librosa.load(file_path, sr=None)
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        stft = np.abs(librosa.stft(audio))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr).T, axis=0)
        result = np.hstack((result, mel))
    return result

# ========== Record and Predict ==========
def record_and_predict():
    status_label.config(text="Recording...")
    window.update()

    # Record audio (3 seconds)
    fs = 22050
    duration = 3
    filename = "recorded.wav"
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    write(filename, fs, recording)

    status_label.config(text="Processing...")
    window.update()

    try:
        features = extract_features(filename)
        features_scaled = scaler.transform([features])
        prediction = model.predict(features_scaled)
        predicted_class = label_classes[np.argmax(prediction)]
        
        status_label.config(text="Ready")
        messagebox.showinfo("Prediction", f"{predicted_class.capitalize()}")

    except Exception as e:
        status_label.config(text="Error")
        messagebox.showerror("Error", str(e))

# ========== GUI ==========
window = tk.Tk()
window.title("Cry Detection")
window.geometry("400x200")
window.configure(bg='lightgray')

status_label = tk.Label(window, text="Ready", font=("Arial", 12), bg='lightgray')
status_label.pack(pady=20)

record_button = tk.Button(window, text="Record and Predict", font=("Arial", 12), command=record_and_predict)
record_button.pack(pady=10)

window.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
